In [272]:
import pandas as pd
import numpy as np

In [273]:
df = pd.read_csv('../MNH TTO Data/MNH TTO Data - MERGED.csv', na_values=' ')

In [274]:
df

,Date,Bed no.,Case no.,Can close,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),...,Timing of Dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Weekday Count,Weekend,Weekend Count,NOT Ward 9
0,01-01-21,905,4020095917,839,4,828,pbh,840,NaN,PBH,...,8:40 AM,1,0,1,0,TRUE,1,FALSE,0,0
1,01-01-21,1202,4020097017,934,8,904,850,NaN,wait for 2nd rx,##############################################...,...,#VALUE!,0,0,0,0,TRUE,1,FALSE,0,1
2,01-01-21,1224,4020095632,934,1,901,905,920,NaN,0:04,...,9:20 AM,1,0,1,0,TRUE,1,FALSE,0,1
3,01-01-21,818,4020097033,934,1,900,901,915,NaN,0:01,...,9:15 AM,1,0,1,0,TRUE,1,FALSE,0,1
4,01-01-21,803,4020097026,1029,1,908,915,1027,"can close, added on metformin and glipizide. 1...",0:07,...,10:27 AM,0,0,0,1,TRUE,1,FALSE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1311,1,1300,1300,1310,add on TTO,0:00,...,1:10 PM,1,0,1,0,TRUE,1,FALSE,0,1
15518,31-12-21,1205,4021107518,1351,1,1335,pbh,1347,NaN,PBH,...,1:47 PM,1,0,1,0,TRUE,1,FALSE,0,1
15519,31-12-21,1131,4021107644,1426,3,1408,1417,1434,NaN,0:09,...,2:34 PM,0,1,1,0,TRUE,1,FALSE,0,1
15520,31-12-21,622,4021108689,1446,2,1359,1409,1441,1431 pt in bathroom,0:10,...,2:41 PM,0,0,0,1,TRUE,1,FALSE,0,1


In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 23 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   Date                                                    15521 non-null  object
 1   Bed no.                                                 15512 non-null  object
 2   Case no.                                                15516 non-null  object
 3   Can close                                               15454 non-null  object
 4   No. of drugs                                            15520 non-null  object
 5   Time TTO received (24 hr format) (eg 1340)              15517 non-null  object
 6   Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)  15508 non-null  object
 7   Timing of Dispensing (24 hr format)                     15512 non-null  object
 8   Comments (for delayed cases) (eg. Pt sleeping 

# Drop columns that should be ignore

In [276]:
# drop repeated column
df.drop(['Can close', 'Time TTO received (24 hr format) (eg 1340)', 'Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)', 'Timing of Dispensing (24 hr format)',
        'Weekday', 'Weekday Count', 'Weekend', 'Weekend Count', 'NOT Ward 9'], axis=1, inplace=True)

In [277]:
#df.drop(df.iloc[:, 18:23], inplace = True, axis = 1)

# Rename common columns

In [278]:
#rename columns
df.rename({'Case no.': 'Case Number', 
            'Bed no.': 'Bed Number', 
            'Timing of packing': 'Time TTO packing', 
            'Timing of Dispensing': 'Time TTO dispensing'},   
           axis=1, inplace=True)

# Handle missing values

In [279]:
df.isnull().sum()

Date                                                      1
Bed Number                                               10
Case Number                                               6
No. of drugs                                              2
Comments (for delayed cases) (eg. Pt sleeping etc)    11665
Time Taken To Pack (mins)                                 1
Time Taken to Reach Pt (mins)                             0
Time TTO received                                         0
Time TTO packing                                          0
Time TTO dispensing                                       0
20 min and below                                          0
Between 20 to 30 min                                      0
30min and below                                           0
More than 30 min                                          0
dtype: int64

In [280]:
#df.iloc[5750:5770]

### Handle missing date

In [281]:
missing_date = df[df['Date'].isnull()]
missing_date

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
8349,NaN,911,4021053861,2,NaN,0:11,0:28,7:54 AM,8:05 AM,8:22 AM,0,1,1,0


In [282]:
# fill up with 15-07-21 based on the given data CSV
df[['Date']] = df[['Date']].fillna('15-07-21')

### Handle missing Time TTO received

In [283]:
# drop rows with missing Time TTO received -> 1 row only
df.replace('#VALUE!', np.nan, inplace = True)
df = df.dropna(subset=['Time TTO received'])
#df.replace('blank', np.nan, inplace = True)

### Handle missing bed number

In [284]:
# handle those with missing bed number, case no by replacing with blank
#missing_bed_no = df[df['Bed Number'].isnull()]
df[['Bed Number']] = df[['Bed Number']].fillna('blank')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing case number

In [285]:
df['Case Number'] = df['Case Number'].replace('  ', np.nan)
df[['Case Number']] = df[['Case Number']].fillna('blank')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Handle missing Time Taken To Pack (mins)

In [286]:
# handle row with missing Time Taken To Pack (mins) by filling up with PBH 
df[df['Time Taken To Pack (mins)'].isnull()]

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
10040,25-08-21,1105,4021069998,1,NaN,NaN,0:17,9:39 AM,PBH,9:56 AM,1,0,1,0


In [287]:
df[['Time Taken To Pack (mins)']] = df[['Time Taken To Pack (mins)']].fillna('PBH')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing Time TTO packing

In [288]:
df[df['Time TTO packing'].isnull()].head(3)

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
506,14-01-21,819,4021001825,1,NaN,blank,0:12,9:48 AM,NaN,10:00 AM,1,0,1,0
885,22-01-21,203,4021005352,6,NaN,blank,blank,3:54 PM,NaN,NaN,0,0,0,0
1324,02-02-21,cot 926,4021008672,1,845 dr in room,blank,2:09,7:26 AM,NaN,9:35 AM,0,0,0,1


In [289]:
df[['Time TTO packing']] = df[['Time TTO packing']].fillna('blank')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing Time TTO dispensing

In [290]:
df[df['Time TTO dispensing'].isnull()].head(3)

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,NaN,0,0,0,0
885,22-01-21,203,4021005352,6,NaN,blank,blank,3:54 PM,blank,NaN,0,0,0,0
2462,02-03-21,1209,4021015997,4,NaN,0:10,blank,9:30 AM,9:40 AM,NaN,0,0,0,0


In [291]:
# handle row with missing Time TTO packing, Time TTO dispensing with blank because it is empty but dont want to drop
df[['Time TTO dispensing']] = df[['Time TTO dispensing']].fillna('blank')
#df[df['Time TTO packing'].isnull()].head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [292]:
df.isnull().sum()

Date                                                      0
Bed Number                                                0
Case Number                                               0
No. of drugs                                              0
Comments (for delayed cases) (eg. Pt sleeping etc)    11659
Time Taken To Pack (mins)                                 0
Time Taken to Reach Pt (mins)                             0
Time TTO received                                         0
Time TTO packing                                          0
Time TTO dispensing                                       0
20 min and below                                          0
Between 20 to 30 min                                      0
30min and below                                           0
More than 30 min                                          0
dtype: int64

In [293]:
df

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,blank,0,0,0,0
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0
15518,31-12-21,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0
15519,31-12-21,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0
15520,31-12-21,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1


In [294]:
#no need to handle missing values for comments since they can leave comments or not

# Handle rows with special characters

### Handle non numeric value in No. of drugs

In [296]:
#select rows not numeric
df[df['No. of drugs'].apply(lambda x: not x.isnumeric())]

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
952,24-01-21,1012,4021006560,-,patient rejected meds,0:34,0:59,10:36 AM,11:10 AM,11:35 AM,0,0,0,1
3560,26-03-21,COT928,4021023823,relabeling,NaN,blank,1:13,7:41 AM,blank,8:54 AM,0,0,0,1


In [297]:
# replace with Unknown number 
df['No. of drugs'] = df['No. of drugs'].str.replace('-','Unknown')
df['No. of drugs'] = df['No. of drugs'].str.replace('relabeling','Unknown')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Handle #### value in Time Taken To Pack (mins)

In [301]:
df_special = df.loc[df['Time Taken To Pack (mins)'].str.contains(r'#') == True]
df_special.head(10)

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,blank,0,0,0,0
284,09-01-21,626,4020093087,2,NaN,##############################################...,0:40,9:40 AM,8:48 AM,10:20 AM,0,0,0,1
341,10-01-21,p08,4021002120,3,NaN,##############################################...,0:18,12:13 PM,11:54 AM,12:31 PM,1,0,1,0
345,10-01-21,1214,4020096026,8,NaN,##############################################...,0:43,2:09 PM,2:02 PM,2:52 PM,0,0,0,1
474,13-01-21,P05,4021001516,9,"long TTO, CD",##############################################...,##############################################...,3:40 PM,2:59 PM,3:34 PM,1,0,1,0
492,14-01-21,931,4020083973,2,NaN,##############################################...,0:28,8:52 AM,6:29 AM,9:20 AM,0,1,1,0
573,15-01-21,621,4021004080,4,1545 can go dispense,##############################################...,##############################################...,3:51 PM,3:25 PM,3:45 PM,1,0,1,0
785,20-01-21,478,4021005600,7,NaN,##############################################...,##############################################...,5:48 PM,4:08 PM,4:29 PM,1,0,1,0
1069,27-01-21,604,4021005370,5,sn will call when pt back from ot,##############################################...,0:20,9:50 AM,8:53 AM,10:10 AM,1,0,1,0
1214,30-01-21,731,4021007676,1,NaN,##############################################...,0:36,10:29 AM,10:25 AM,11:05 AM,0,0,0,1


In [302]:
# assuming those with ### are PBH TTO because packing time is earlier than TTO received --> to check with Qi En

df['Time Taken To Pack (mins)'] = df['Time Taken To Pack (mins)'].str.replace('#','PBH')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [303]:
df

,Date,Bed Number,Case Number,No. of drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO received,Time TTO packing,Time TTO dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0
1,01-01-21,1202,4020097017,8,wait for 2nd rx,PBHPBHPBHPBHPBHPBHPBHPBHPBHPBHPBHPBHPBHPBHPBHP...,blank,9:04 AM,8:50 AM,blank,0,0,0,0
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0
15518,31-12-21,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0
15519,31-12-21,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0
15520,31-12-21,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1


In [164]:
# drop rows with ######### value in Time Taken to Reach Pt (mins) metric
#df[df.iloc[:, 5:6].contains(r'[^0-9a-zA-Z]').index]
#df_special = df.loc[df['Time Taken To Pack (mins)'].str.contains(r'#') == True]
#df = df.loc[df['Time Taken to Reach Pt (mins)'].str.contains(r'#') == False]
#df_special

In [304]:
df.to_csv("MNH TTO TAT (cleaned).csv", index = None, header=True)

# EDA

In [305]:
df = pd.read_csv('MNH TTO TAT (cleaned).csv')

In [306]:
df.describe()

,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
count,15513.000000,15513.000000,15513.000000,15513.000000
mean,0.232901,0.231548,0.464449,0.534906
std,0.422694,0.421835,0.498751,0.498796
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000
